In [1]:
import os
import pandas as pd

# Configuración
folder_path = 'kahoot'  # Cambia esto a la ruta de tu carpeta
question_time = 20
columns = ['Question', 'Answer 1', 'Answer 2', 'Answer 3', 'Answer 4', 'Correct Answers']
max_rows_per_file = 100

# Función para asignar la respuesta
def assign_answer(row):
    answers = ['Answer 1', 'Answer 2', 'Answer 3', 'Answer 4']
    for i, col in enumerate(answers, start=1):
        if row[col] == row['Correct Answers']:
            return i

def read_excel_file(path):
    # Cargar datos desde el archivo Excel
    data = pd.read_excel(path, sheet_name='RawReportData Data')

    # Eliminar duplicados basados en la columna 'Question' y resetear el índice
    unique_data = data.drop_duplicates(subset='Question', keep='first').reset_index(drop=True)

    # Seleccionar columnas relevantes y agregar la columna 'tiempo'
    processed_df = unique_data[columns].copy()
    processed_df['tiempo'] = question_time

    # Crear la nueva columna 'respuesta' con la función assign_answer
    processed_df['respuesta'] = processed_df.apply(assign_answer, axis=1)

    # Eliminar la columna 'Correct Answers' y las filas con valores NaN en 'respuesta'
    final_df = processed_df.drop(['Correct Answers'], axis=1).dropna(subset=['respuesta']).reset_index(drop=True)

    # Convertir la columna 'respuesta' a enteros sin decimales y renombrar a 'Correct Answers'
    final_df['respuesta'] = final_df['respuesta'].astype(int)
    final_df.rename(columns={'respuesta': 'Correct Answers'}, inplace=True)

    return final_df

def split_dataframe(folder):
    # Lista para almacenar DataFrames de cada archivo
    dfs_list = []

    # Recorrer archivos en la carpeta
    for file in os.listdir(folder):
        if file.endswith('.xlsx'):
            file_path = os.path.join(folder, file)
            df = read_excel_file(file_path)
            dfs_list.append(df)

    # Concatenar todos los DataFrames en uno solo
    combined_df = pd.concat(dfs_list, ignore_index=True)

    # Eliminar duplicados basados en la columna 'Question' y resetear el índice
    unique_combined_df = combined_df.drop_duplicates(subset='Question', keep='first').reset_index(drop=True)

    # Calcula la cantidad total de archivos que se crearán
    total_files = (len(unique_combined_df) - 1) // max_rows_per_file + 1

    # Divide el DataFrame original en varios DataFrames más pequeños
    small_dataframes = [unique_combined_df.iloc[i * max_rows_per_file: (i + 1) * max_rows_per_file] for i in range(total_files)]

    # Guarda cada DataFrame más pequeño en un nuevo archivo Excel
    for i, small_df in enumerate(small_dataframes):
        new_file_name = f'split_file_{i+1}.xlsx'
        small_df.to_excel(new_file_name, index=False)
        print(f'Se creó el archivo: {new_file_name}')

split_dataframe(folder_path)


ValueError: Worksheet named 'RawReportData Data' not found